# Initial EDA (R)

## Read in data

In [1]:
library(tidyverse)
library(data.table)
library(glue)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse




In [2]:
awards <- fread('raw_data/awards.csv')

In [3]:
glimpse(awards)

Rows: 11,256
Columns: 7
$ awardDate         <date> 2017-12-21, 2017-12-20, 2017-12-20, 2017-12-20, 201…
$ awardSeason       <int> 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017…
$ awardId           <chr> "WARRENSPAHN", "MILBORGAS", "MILBORGAS", "MILBORGAS"…
$ awardName         <chr> "Warren Spahn Award", "MiLB.com Organization All-Sta…
$ playerId          <int> 477132, 474319, 592530, 593833, 600466, 641914, 6453…
$ playerName        <chr> "Clayton Kershaw", "Brandon Snyder", "Jose Marmolejo…
$ awardPlayerTeamId <int> 119, 120, 120, 120, 120, 120, 120, 120, 120, 671, 67…


In [5]:
print(glue("was using: {getDTthreads()} threads"))
setDTthreads(parallel::detectCores()-1)
print(glue("now using: {getDTthreads()} threads"))


using: 4 threads
now using: 7 threads


In [6]:
players <- fread('raw_data/players.csv')

In [57]:
train <- read_csv('raw_data/train.csv',n_max=10,col_types = cols(.default='c'))


`nextDayPlayerEngagement`, `playerTwitterFollowers`, `teamTwitterFollowers` in JSON format

In [58]:
glimpse(train)

Rows: 10
Columns: 12
$ date                    <chr> "20180101", "20180102", "20180103", "20180104"…
$ nextDayPlayerEngagement <chr> "[{\"engagementMetricsDate\":\"2018-01-02\",\"…
$ games                   <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA
$ rosters                 <chr> "[{\"playerId\":400121,\"gameDate\":\"2018-01-…
$ playerBoxScores         <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA
$ teamBoxScores           <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA
$ transactions            <chr> "[{\"transactionId\":340732,\"playerId\":54734…
$ standings               <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA
$ awards                  <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA
$ events                  <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA
$ playerTwitterFollowers  <chr> "[{\"date\":\"2018-01-01\",\"playerId\":545361…
$ teamTwitterFollowers    <chr> "[{\"date\":\"2018-01-01\",\"teamId\":147,\"te…


In [59]:
map_df(train$nextDayPlayerEngagement, function(x) jsonlite::fromJSON(x)) %>%
       glimpse

Rows: 20,610
Columns: 6
$ engagementMetricsDate <chr> "2018-01-02", "2018-01-02", "2018-01-02", "2018-…
$ playerId              <int> 628317, 547989, 519317, 607625, 592547, 641553, …
$ target1               <dbl> 0.011167071, 0.042993222, 0.974326905, 0.0067002…
$ target2               <dbl> 4.4747082, 5.5933852, 56.1770428, 2.6750973, 0.6…
$ target3               <dbl> 5.167730e-03, 4.503307e-02, 1.369375e+01, 5.1677…
$ target4               <dbl> 5.7352941, 2.7941176, 64.1666667, 1.8627451, 0.9…


Read in the csv chunk-wise, formatting the JSON columns as we go...

In [66]:
read_nested_cols <- function(file, column_name, chunk_size=5){
    cb <- function(x, pos) map_df(x[[column_name]], function(x) if(jsonlite::validate(x)) jsonlite::fromJSON(x))
                                  
    read_csv_chunked(file ,chunk_size = chunk_size, callback = DataFrameCallback$new(cb),col_types = cols(.default='c'))
}




In [41]:
nextDayPlayerEngagement <- read_nested_cols('raw_data/train.csv', 'nextDayPlayerEngagement', 5 )


── Column specification ────────────────────────────────────────────────────────
cols(
  date = col_double(),
  nextDayPlayerEngagement = col_character(),
  games = col_logical(),
  rosters = col_character(),
  playerBoxScores = col_logical(),
  teamBoxScores = col_logical(),
  transactions = col_character(),
  standings = col_logical(),
  awards = col_logical(),
  events = col_logical(),
  playerTwitterFollowers = col_character(),
  teamTwitterFollowers = col_character()
)




In [42]:
glimpse(nextDayPlayerEngagement)

Rows: 2,506,176
Columns: 6
$ engagementMetricsDate <chr> "2018-01-02", "2018-01-02", "2018-01-02", "2018-…
$ playerId              <int> 628317, 547989, 519317, 607625, 592547, 641553, …
$ target1               <dbl> 0.011167071, 0.042993222, 0.974326905, 0.0067002…
$ target2               <dbl> 4.4747082, 5.5933852, 56.1770428, 2.6750973, 0.6…
$ target3               <dbl> 5.167730e-03, 4.503307e-02, 1.369375e+01, 5.1677…
$ target4               <dbl> 5.7352941, 2.7941176, 64.1666667, 1.8627451, 0.9…


We can loop through all of the remaining columns and write out to a fast format for future use...

In [80]:
read_nested_cols('raw_data/train.csv', 'games', 5 )

In [91]:
unnested <- list()
for(i in names(train)){
    df <- read_nested_cols('raw_data/train.csv', i, 5 )
    feather::write_feather(df, glue("raw_data/train_{i}.feather"))
}

unnested

[1] "date"
[1] "nextDayPlayerEngagement"
[1] "games"
[1] "rosters"
[1] "playerBoxScores"
[1] "teamBoxScores"
[1] "transactions"
[1] "standings"
[1] "awards"
[1] "events"
[1] "playerTwitterFollowers"
[1] "teamTwitterFollowers"


In [ ]:
unnested$games